In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *
from filepath import *

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from json import loads

# import spacy
from spacy.lang.id import Indonesian
nlp = Indonesian()  # use directly
stopwords = spacy.lang.id.stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","jurus","the","of"}

In [2]:
# #Akronim
def slang(tokenized_sentence):
    slang_word_dict = loads(open("slang_word_dict.txt", 'r').read())

    for index in range(len(tokenized_sentence)):
        for key, value in slang_word_dict.items():
            for v in value:
                if tokenized_sentence[index] == v:
                    tokenized_sentence[index] = key
                else:
                    continue
                    
    return " ".join(tokenized_sentence)

def preprocessing(text):
    text = re.sub('<[^<]+?>', '', str(text)) #remove tag
    text = text.lower() #lower\n",
    text = re.sub(r'[^a-z]',' ',str(text)) #get alphabet only
    text = re.sub(r'\s+', ' ', text) #remove white space
    text = re.sub(r'sobat pintar','',text) # sorry:(
    text = [token.text for token in nlp(text)] #Token
    text = slang(text)#slang word
    text = re.sub(r'\s+', ' ', text) #remove white space
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return text

def get_data():
    data = open_table(['entryId','content'],'BlogsEntry')
    
    return data

In [3]:
#get data
data = get_data()
data = rename_column(data,{0:'entryId', 1:'content'})
data.content = data.content.apply(preprocessing)
data.head()

,entryId,content
0,204420,"[kakak, blog, www, herniawan, com, bahas, tips..."
1,288433,"[luang, snmptn, nilai, rapormu, turun, ombak, ..."
2,288455,"[tarik, snmptn, hmm, akibat, nilai, rapor, flu..."
3,288478,"[dengar, ajar, muncul, dikepalamu, kesal, mala..."
4,291921,"[masuk, kelas, suai, jadwal, pulang, sekolah, ..."


In [4]:
#Save
data.to_csv('../data/data_berita_clean.csv', sep=',',header=data.columns.tolist(),index=None)

In [5]:
#Data Shape
len(data.entryId.unique())

745

In [7]:
# #Count Tag
# tag = data.tag.value_counts()
# display(tag)

# df = pd.DataFrame(tag)
# # df.head()
# #Save
# df.to_csv('tag.csv', sep=',',header=df.columns.tolist())